<a href="https://colab.research.google.com/github/imeunu/Capstone_PBL/blob/main/Speech_Recognition/ASR_10class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 10 classes
- no preprocessing
- only mfcc
- cnn with 29,990 prams


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import re
import csv
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers,regularizers
from tensorflow.keras.models import Sequential, load_model, model_from_json 
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dense,Activation,Flatten,GlobalAveragePooling2D,GlobalMaxPooling2D,Dropout,AveragePooling2D,BatchNormalization, Softmax
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.python.framework.ops import disable_eager_execution
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from tensorflow.python.client import device_lib

In [3]:
X_data= np.load('/content/drive/MyDrive/X_10_data.npy')
Y_data= np.load('/content/drive/MyDrive/Y_10_data.npy')

In [4]:
Y_data=Y_data.reshape(-1,)
keywords = ('yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop','go')

In [5]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y= train_test_split(X_data, Y_data, test_size= 0.2, stratify= Y_data)

train_X= np.array(train_X)
test_X= np.array(test_X)
train_Y= np.array(train_Y)
test_Y= np.array(test_Y)
train_Y = to_categorical(train_Y, len(keywords))
test_Y = to_categorical(test_Y, len(keywords))

In [6]:
save_path= '/content/drive/MyDrive/Speech_recognition/result'

In [7]:

#Model
inp_shape = train_X.shape[1:]
def generate_model_cnn(mode='words', pooling=0, layers=3, kernel=11):
    model = Sequential()
    print("=================================CNN=====================================")
    model.add(Conv2D(filters=30, kernel_size=(3,3), strides = 1, padding ='same', input_shape=inp_shape, activation = 'relu')) #add convolution layer
    model.add(AveragePooling2D(pool_size=(3,3), strides=2)) #pooling layer
    model.add(Conv2D(filters=30, kernel_size=(3,3), strides = 1, padding ='same', activation = 'relu'))
    model.add(AveragePooling2D(pool_size=(3,3), strides=None))
    model.add(Flatten())
    model.add(Dense(50, activation = 'relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.1))
    model.add(Dense(10, activation = 'softmax'))
    return model

def generate_model(mode='words', summary=True):
    model = generate_model_cnn(mode=mode, pooling=0)
    if summary:
        model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

callbacks_list  = [
    EarlyStopping( monitor = 'loss',  min_delta=0.001, patience=100,
                  verbose=1, mode='auto'),
    ModelCheckpoint(filepath = save_path+"/weights.{epoch:02d}-{val_loss:.3f}-{val_accuracy:.2f}.hdf5",
                    monitor = 'val_loss',
                    save_best_only=True ),
    ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=7, verbose=1, min_delta=1e-4)]

model = generate_model(mode='words',summary=True)
history = model.fit(train_X, train_Y, epochs=300, batch_size=100, validation_data=(test_X,test_Y),callbacks=callbacks_list)

=================================CNN=====================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 44, 30)        300       
_________________________________________________________________
average_pooling2d (AveragePo (None, 6, 21, 30)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 21, 30)         8130      
_________________________________________________________________
average_pooling2d_1 (Average (None, 2, 7, 30)          0         
_________________________________________________________________
flatten (Flatten)            (None, 420)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                21050     
________________________________________________